In [487]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve,classification_report,confusion_matrix
from sklearn.preprocessing import MinMaxScaler, StandardScaler
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [75]:
import lightgbm as lgb

In [690]:
import catboost as catb

In [760]:
# Считывание данных

sot = pd.read_csv('data\sotrudniki.csv', sep = ';')
rod = pd.read_csv('rodstvenniki.csv', sep = ';')
ogrv = pd.read_csv('data\OGRV.csv', sep = ';')

In [761]:
train_full=sot.copy()
train_num=train_full.drop(['date', 'sick', 'home_to_work_distance', 'home_to_work_duration', 'gender', 'category', \
                       'name_post_lvl4', 'name_post_lvl5', 'prof_post_augment', \
                      'name_fact_lvl4','name_fact_lvl5','prof_fact_augment','razryad_fact','married', 'education'], axis=1)


In [762]:
train_num=train_num.groupby('hash_tab_num').max()


In [763]:
train_num['work_experience_company']=train_num['work_experience_company'].fillna(train_num['work_experience_company'].mean())
train_num['work_experience_factory']=train_num['work_experience_factory'].fillna(train_num['work_experience_factory'].mean())
train_num['work_experience_all']=train_num['work_experience_all'].fillna(train_num['work_experience_all'].mean())

In [764]:
train_num.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2653 entries, 0 to 2652
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   date_of_birth            2653 non-null   int64  
 1   is_local                 2653 non-null   int64  
 2   razryad_post             2653 non-null   int64  
 3   child_num                2653 non-null   int64  
 4   work_experience_company  2653 non-null   float64
 5   work_experience_factory  2653 non-null   float64
 6   work_experience_all      2653 non-null   float64
dtypes: float64(3), int64(4)
memory usage: 165.8 KB


In [765]:
train_obj=train_full[['hash_tab_num','gender', 'category', \
                       'name_post_lvl4', 'name_post_lvl5', 'prof_post_augment', \
                      'name_fact_lvl4','name_fact_lvl5','prof_fact_augment','razryad_fact','married', 'education']]

In [766]:
train_obj=train_obj.groupby('hash_tab_num').first()

In [767]:
risk_post_lvl4=['Отделение_6', 'Отделение_9']
train_obj['in_risk_post_lvl4']=0
train_obj.loc[train_obj.name_post_lvl4.isin(risk_post_lvl4), 'in_risk_post_lvl4']=1

In [768]:
risk_post_lvl5=['передел 11', 'передел 15', 'передел 5']
train_obj['in_risk_post_lvl5']=0
train_obj.loc[train_obj.name_post_lvl5.isin(risk_post_lvl5), 'in_risk_post_lvl5']=1

In [769]:
train_obj=train_obj.drop(['name_post_lvl4', 'name_post_lvl5', 'name_fact_lvl4', 'name_fact_lvl5'], axis=1)
train_obj=train_obj.drop(['prof_post_augment'], axis=1)

In [770]:
train_obj.loc[train_obj.category.isnull(), 'category']='Рабочие'

In [771]:
train_obj.loc[train_obj.education.isnull(), 'education']='Нач.профессиональное'

In [772]:
train_obj.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2653 entries, 0 to 2652
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   gender             2653 non-null   object
 1   category           2653 non-null   object
 2   prof_fact_augment  2653 non-null   object
 3   razryad_fact       2653 non-null   int64 
 4   married            2389 non-null   object
 5   education          2653 non-null   object
 6   in_risk_post_lvl4  2653 non-null   int64 
 7   in_risk_post_lvl5  2653 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 186.5+ KB


In [773]:
train_full=train_num.merge(train_obj, how='left', left_on=train_num.index, right_on=train_obj.index)

In [774]:
# заменить на заполнение от кол-ва детей
train_full.loc[train_full.married.isnull(), 'married']='жен/зм'

In [775]:
train_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2653 entries, 0 to 2652
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   key_0                    2653 non-null   int64  
 1   date_of_birth            2653 non-null   int64  
 2   is_local                 2653 non-null   int64  
 3   razryad_post             2653 non-null   int64  
 4   child_num                2653 non-null   int64  
 5   work_experience_company  2653 non-null   float64
 6   work_experience_factory  2653 non-null   float64
 7   work_experience_all      2653 non-null   float64
 8   gender                   2653 non-null   object 
 9   category                 2653 non-null   object 
 10  prof_fact_augment        2653 non-null   object 
 11  razryad_fact             2653 non-null   int64  
 12  married                  2653 non-null   object 
 13  education                2653 non-null   object 
 14  in_risk_post_lvl4       

In [776]:
#### количество часов, отработанных в разных сменах

In [777]:
ogrv_train=ogrv.copy()

In [778]:
ogrv_train['number_of_working_hours'] = ogrv_train['number_of_working_hours'].str.replace(',','.')

ogrv_train['number_of_working_hours'] = ogrv_train['number_of_working_hours'].astype(float)



In [779]:
ogrv_train = ogrv_train.groupby(['hash_tab_num','work_shift_type']).number_of_working_hours.sum()

In [780]:
ogrv_train=pd.DataFrame(ogrv_train.reset_index().fillna(0))

In [781]:
ogrv_train=ogrv_train.pivot(index='hash_tab_num', columns='work_shift_type', values='number_of_working_hours')

In [782]:
ogrv_train.fillna(0,inplace=True)

In [783]:
train_full=train_full.merge(ogrv_train, how='left', right_on=ogrv_train.index, left_on= 'key_0' )
train_full

,key_0,date_of_birth,is_local,razryad_post,child_num,work_experience_company,work_experience_factory,work_experience_all,gender,category,prof_fact_augment,razryad_fact,married,education,in_risk_post_lvl4,in_risk_post_lvl5,ВЫХ,Смена 1,Смена 2,Смена 3
0,0,1985,1,5,1,13.0,13.0000,14.000000,мужской,Рабочие,Должность_0,3,хол/нз,Среднее(полное)общее,0,0,0.0,186.65,1505.50,106.70
1,1,1983,1,5,1,14.0,14.0000,16.000000,мужской,Рабочие,Должность_4,5,жен/зм,Нач.профессиональное,0,0,0.0,141.00,1564.70,187.50
2,2,1967,1,0,0,14.0,14.0000,29.000000,женский,Руководители,Должность_5,0,жен/зм,Высшее образование,0,0,0.0,1183.25,4915.15,257.15
3,3,1976,0,4,1,12.0,12.0000,15.000000,мужской,Рабочие,Должность_6,4,хол/нз,Нач.профессиональное,0,0,NaN,NaN,NaN,NaN
4,4,1986,1,5,1,12.0,12.0000,14.000000,мужской,Рабочие,Должность_7,5,жен/зм,Нач.профессиональное,0,0,0.0,0.00,120.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2648,2648,1980,1,0,2,12.0,11.9572,15.829247,мужской,Рабочие,Должность_19,0,жен/зм,Среднее(полное)общее,0,0,0.0,88.00,96.00,56.00
2649,2649,1982,0,0,1,12.0,11.9572,15.829247,мужской,Рабочие,Должность_19,0,разв.,Нач.профессиональное,0,0,0.0,72.00,61.10,96.00
2650,2650,1994,1,0,0,12.0,11.9572,15.829247,мужской,Рабочие,Должность_19,0,хол/нз,Среднее(полное)общее,0,0,0.0,0.00,228.40,7.50
2651,2651,1998,1,0,0,12.0,11.9572,15.829247,мужской,Рабочие,Должность_32,0,хол/нз,Основное общее,0,0,0.0,67.50,22.50,37.50


In [784]:
train_full.rename(columns={'Смена 1':'smena_1','Смена 2':'smena_2', 'Смена 3':'smena_3' }, inplace=True)

In [785]:
train_full.fillna(0, inplace=True)

In [786]:
train_full.drop(['ВЫХ'], inplace=True, axis=1)

In [787]:
###частота ухода на больничный

In [788]:
train_q=sot.copy()
train_q

,hash_tab_num,date,date_of_birth,is_local,gender,category,name_post_lvl4,name_post_lvl5,prof_post_augment,razryad_post,name_fact_lvl4,name_fact_lvl5,prof_fact_augment,razryad_fact,married,child_num,work_experience_company,work_experience_factory,work_experience_all,education,sick,home_to_work_distance,home_to_work_duration
0,0,2015-04-01,1985,1,мужской,Рабочие,NaN,NaN,Должность_0,3,NaN,NaN,Должность_0,3,хол/нз,1,9.0,9.0,10.0,Среднее(полное)общее,0,NaN,NaN
1,0,2015-05-01,1985,1,мужской,Рабочие,NaN,NaN,Должность_0,3,NaN,NaN,Должность_0,3,хол/нз,1,9.0,9.0,10.0,Среднее(полное)общее,0,NaN,NaN
2,0,2015-06-01,1985,1,мужской,Рабочие,NaN,NaN,Должность_0,3,NaN,NaN,Должность_0,3,хол/нз,1,9.0,9.0,10.0,Среднее(полное)общее,0,NaN,NaN
3,0,2015-07-01,1985,1,мужской,Рабочие,NaN,NaN,Должность_0,3,NaN,NaN,Должность_0,3,хол/нз,1,9.0,9.0,10.0,Среднее(полное)общее,0,NaN,NaN
4,0,2015-08-01,1985,1,мужской,Рабочие,NaN,NaN,Должность_0,3,NaN,NaN,Должность_0,3,хол/нз,1,9.0,9.0,10.0,Среднее(полное)общее,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99209,2648,2019-08-01,1980,1,мужской,NaN,NaN,NaN,NaN,0,NaN,NaN,Должность_19,0,жен/зм,2,NaN,NaN,NaN,Среднее(полное)общее,0,4953.5,6.878333
99210,2649,2019-08-01,1982,0,мужской,NaN,NaN,NaN,NaN,0,NaN,NaN,Должность_19,0,разв.,1,NaN,NaN,NaN,Нач.профессиональное,0,NaN,NaN
99211,2650,2019-08-01,1994,1,мужской,NaN,NaN,NaN,NaN,0,NaN,NaN,Должность_19,0,хол/нз,0,NaN,NaN,NaN,Среднее(полное)общее,0,NaN,NaN
99212,2651,2019-08-01,1998,1,мужской,Рабочие,Отделение_8,передел 7,Должность_56,0,Отделение_8,передел 7,Должность_32,0,хол/нз,0,NaN,NaN,NaN,Основное общее,1,2463.8,3.113333


In [789]:
sick_rate=pd.DataFrame(train_q.groupby('hash_tab_num').sum().sick)

In [790]:
sick_rate['work_time']=train_q.groupby('hash_tab_num').count().date

In [791]:
sick_rate['sick_rate']=sick_rate['sick']/sick_rate['work_time']

In [792]:
sick_rate.rename({'sick':'sick_month'}, inplace=True, axis=1)

In [793]:
train_full=train_full.merge(sick_rate, how='left', left_on='key_0', right_on=sick_rate.index)

In [794]:
train_full

,key_0,date_of_birth,is_local,razryad_post,child_num,work_experience_company,work_experience_factory,work_experience_all,gender,category,prof_fact_augment,razryad_fact,married,education,in_risk_post_lvl4,in_risk_post_lvl5,smena_1,smena_2,smena_3,sick_month,work_time,sick_rate
0,0,1985,1,5,1,13.0,13.0000,14.000000,мужской,Рабочие,Должность_0,3,хол/нз,Среднее(полное)общее,0,0,186.65,1505.50,106.70,15,53,0.283019
1,1,1983,1,5,1,14.0,14.0000,16.000000,мужской,Рабочие,Должность_4,5,жен/зм,Нач.профессиональное,0,0,141.00,1564.70,187.50,6,53,0.113208
2,2,1967,1,0,0,14.0,14.0000,29.000000,женский,Руководители,Должность_5,0,жен/зм,Высшее образование,0,0,1183.25,4915.15,257.15,0,53,0.000000
3,3,1976,0,4,1,12.0,12.0000,15.000000,мужской,Рабочие,Должность_6,4,хол/нз,Нач.профессиональное,0,0,0.00,0.00,0.00,3,53,0.056604
4,4,1986,1,5,1,12.0,12.0000,14.000000,мужской,Рабочие,Должность_7,5,жен/зм,Нач.профессиональное,0,0,0.00,120.00,0.00,7,53,0.132075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2648,2648,1980,1,0,2,12.0,11.9572,15.829247,мужской,Рабочие,Должность_19,0,жен/зм,Среднее(полное)общее,0,0,88.00,96.00,56.00,0,1,0.000000
2649,2649,1982,0,0,1,12.0,11.9572,15.829247,мужской,Рабочие,Должность_19,0,разв.,Нач.профессиональное,0,0,72.00,61.10,96.00,0,1,0.000000
2650,2650,1994,1,0,0,12.0,11.9572,15.829247,мужской,Рабочие,Должность_19,0,хол/нз,Среднее(полное)общее,0,0,0.00,228.40,7.50,0,1,0.000000
2651,2651,1998,1,0,0,12.0,11.9572,15.829247,мужской,Рабочие,Должность_32,0,хол/нз,Основное общее,0,0,67.50,22.50,37.50,1,1,1.000000


In [795]:
train_full['under_22']=0
train_full.loc[train_full.date_of_birth>=1996, 'under_22' ]=1

In [796]:
train_full['22-61']=0
train_full.loc[(train_full.date_of_birth<1996)&(train_full.date_of_birth>=1957), '22-61' ]=1

In [797]:
train_full['over_61']=0
train_full.loc[(train_full.date_of_birth<1957), 'over_61' ]=1

In [798]:
train_full.drop([ 'work_experience_company', 'work_experience_factory'], inplace=True, axis=1)

In [799]:
train_full.drop('key_0', inplace=True, axis=1)

In [800]:
train_full.columns

Index(['date_of_birth', 'is_local', 'razryad_post', 'child_num',
       'work_experience_all', 'gender', 'category', 'prof_fact_augment',
       'razryad_fact', 'married', 'education', 'in_risk_post_lvl4',
       'in_risk_post_lvl5', 'smena_1', 'smena_2', 'smena_3', 'sick_month',
       'work_time', 'sick_rate', 'under_22', '22-61', 'over_61'],
      dtype='object')

In [732]:
train_full

,date_of_birth,is_local,razryad_post,child_num,work_experience_all,gender,category,prof_fact_augment,razryad_fact,married,education,in_risk_post_lvl4,in_risk_post_lvl5,smena_1,smena_2,smena_3,sick_month,work_time,sick_rate,under_22,22-61,over_61,sick
0,1985,1,5,1,14.000000,мужской,Рабочие,Должность_0,3,хол/нз,Среднее(полное)общее,0,0,186.65,1505.50,106.70,15,53,0.283019,0,1,0,15
1,1983,1,5,1,16.000000,мужской,Рабочие,Должность_4,5,жен/зм,Нач.профессиональное,0,0,141.00,1564.70,187.50,6,53,0.113208,0,1,0,6
2,1967,1,0,0,29.000000,женский,Руководители,Должность_5,0,жен/зм,Высшее образование,0,0,1183.25,4915.15,257.15,0,53,0.000000,0,1,0,0
3,1976,0,4,1,15.000000,мужской,Рабочие,Должность_6,4,хол/нз,Нач.профессиональное,0,0,0.00,0.00,0.00,3,53,0.056604,0,1,0,3
4,1986,1,5,1,14.000000,мужской,Рабочие,Должность_7,5,жен/зм,Нач.профессиональное,0,0,0.00,120.00,0.00,7,53,0.132075,0,1,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2648,1980,1,0,2,15.829247,мужской,Рабочие,Должность_19,0,жен/зм,Среднее(полное)общее,0,0,88.00,96.00,56.00,0,1,0.000000,0,1,0,0
2649,1982,0,0,1,15.829247,мужской,Рабочие,Должность_19,0,разв.,Нач.профессиональное,0,0,72.00,61.10,96.00,0,1,0.000000,0,1,0,0
2650,1994,1,0,0,15.829247,мужской,Рабочие,Должность_19,0,хол/нз,Среднее(полное)общее,0,0,0.00,228.40,7.50,0,1,0.000000,0,1,0,0
2651,1998,1,0,0,15.829247,мужской,Рабочие,Должность_32,0,хол/нз,Основное общее,0,0,67.50,22.50,37.50,1,1,1.000000,1,0,0,1


In [801]:
model = catb.CatBoostClassifier()      # parameters not required.
model.load_model('models\prefilter.75')

In [802]:
th=0.43
preds_proba = model.predict_proba(train_full)
preds_proba = (preds_proba[:, 1] > th).astype(np.int)

In [803]:
preds_first_lev=pd.DataFrame(preds_proba, columns=['sick'])
list_to_exclude=preds_first_lev[preds_first_lev.sick==0].index

In [804]:
list_to_exclude

Int64Index([   2,   10,   13,   15,   24,   25,   29,   30,   32,   34,
            ...
            2628, 2634, 2635, 2636, 2637, 2638, 2647, 2649, 2650, 2652],
           dtype='int64', length=699)

### Второй уровень

In [970]:
# Создание вспомогательного датафрейма с информацией о количестве смен сотрудника в месяце

ogrv['month'] = ogrv['date'].map(lambda x: x[0:8] + str('01'))
kolvo_smen = ogrv[ogrv.work_shift_type.isin(['Смена 1', 'Смена 2', 'Смена 3'])]\
[['hash_tab_num','month','work_shift_type']].groupby(['hash_tab_num','month']).agg('count').reset_index()
kolvo_smen.columns = ['hash_tab_num', 'date', 'work_shift_type_count']

In [971]:
# Создание вспомогательного датафрейма с информацией о факте больничного в текущем месяце

kolvo_bolni4 = ogrv[ogrv.graphic_rule_level_1.isin(['Больничный'])]\
[['hash_tab_num','month','graphic_rule_level_1']].groupby(['hash_tab_num','month']).agg('count').reset_index()

kolvo_bolni4['graphic_rule_level_1'] = 1
kolvo_bolni4.columns = ['hash_tab_num', 'date', 'sick']

In [972]:
# Базовый датафремй

sot_data = sot[['hash_tab_num','date','category','gender','razryad_fact','work_experience_company',
                'name_fact_lvl5','education','home_to_work_distance']]
sot_data.gender = sot_data['gender'].map(lambda x: 1 if x == 'мужской' else 0)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [973]:
# Создание вспомогательно датасета с информацией о родственниках - пенсионерах
# (55 лет для женщин и 60 лет для мужчин для региона севера)

sot_data = pd.merge(sot_data,rod, how = 'left', on = 'hash_tab_num')
sot_data['rel_cur_old'] = ([int(x[0:4]) for x in sot_data['date']] - sot_data['rel_birth'])
sot_data['rel_is_male'] = sot_data.rel_type.map(lambda x:1 if x \
    in ['Сын', 'Муж', 'Отец', 'Пасынок', 'Внук','Брат'] else 0)

retiree = sot_data[((sot_data.rel_cur_old > 55) & (sot_data.rel_is_male == 0) \
               | (sot_data.rel_cur_old > 60) & (sot_data.rel_is_male == 1))]\
    [['hash_tab_num','date','rel_is_male']].groupby(['hash_tab_num','date']).agg('count').reset_index()
sot_data.drop(['rel_type','rel_birth','rel_cur_old','rel_is_male'], axis = 1, inplace = True)

In [974]:
# Создание вспомогательно датасета с информацией о количестве сотрудников в подразделении
# по фактическому месту работы

division_count = sot_data[['hash_tab_num','date','name_fact_lvl5']].\
groupby(['name_fact_lvl5','date']).agg('count').reset_index()

division_count.columns = ['name_fact_lvl5', 'date', 'personel_num']

sot_data = pd.merge(sot_data, division_count, how = 'left', on = ['date','name_fact_lvl5'])

In [975]:
# Создание dummy переменных

sot_data.education = sot_data['education']\
.map(lambda x: 'Высшее' if x in ['Высшее образование','Высшее-бакалавриат','Высшее-специалитет'] else(\
'Среднее_профессинальное' if x in ['Ср.профессиональное','Нач.профессиональное'] else 'Начальное_среднее'))
sot_data = pd.get_dummies(sot_data, columns = ['category','education','razryad_fact'])\
.drop('name_fact_lvl5', axis = 1)

In [976]:
# Создание единого датасета для будущего использования в модели

merged_data = pd.merge(sot_data, retiree, how = 'left', on = ['hash_tab_num','date'])
merged_data = pd.merge(merged_data, kolvo_smen, how = 'left', on = ['hash_tab_num','date'])
merged_data = pd.merge(merged_data, kolvo_bolni4, how = 'left', on = ['hash_tab_num','date'])
merged_data = merged_data.drop_duplicates()

In [977]:
# Создание 12ти столбцов с датами будущих периодов для формирования таргетов

merged_data['sick'] = merged_data['sick'].fillna(0)
merged_data['target_dates'] = merged_data['date'].apply(lambda x: pd.date_range((x),\
    periods = 13, freq='1MS',closed = 'right'))
new_target_dates = pd.DataFrame(merged_data['target_dates'].tolist(), \
    columns = ['y_dt_'+str(i) for i in range(1,13)], index = merged_data.index)
merged_data = pd.merge(merged_data,new_target_dates, left_index=True, right_index=True)
merged_data.drop(['target_dates'],axis = 1, inplace = True)
merged_data['date'] = pd.to_datetime(merged_data['date'])

In [978]:
# Присоединение данных о больничных к будущим периодам созданным на предыдущем шаге

for i in range(1,13):
    dt_col_name = 'y_dt_'+str(i)
    y_col_name = 'y_'+str(i)
    
    targets_tmp = merged_data[['date','hash_tab_num','sick']]
    targets_tmp.columns = [dt_col_name, 'hash_tab_num', y_col_name]
    
    merged_data = pd.merge(merged_data, targets_tmp, how = 'left', on = [dt_col_name, 'hash_tab_num'])
    
    merged_data.drop(dt_col_name, axis = 1, inplace = True)

In [979]:
merged_data

,hash_tab_num,date,gender,work_experience_company,home_to_work_distance,personel_num,category_Рабочие,category_Руководители,category_Служащие,category_Специалисты,education_Высшее,education_Начальное_среднее,education_Среднее_профессинальное,razryad_fact_0,razryad_fact_1,razryad_fact_2,razryad_fact_3,razryad_fact_4,razryad_fact_5,razryad_fact_6,rel_is_male,work_shift_type_count,sick,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,y_10,y_11,y_12
0,0,2015-04-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,0,2015-05-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0,2015-06-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,0,2015-07-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,0,2015-08-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99209,2648,2019-08-01,1,NaN,4953.5,NaN,0,0,0,0,0,1,0,1,0,0,0,0,0,0,NaN,19.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99210,2649,2019-08-01,1,NaN,NaN,NaN,0,0,0,0,0,0,1,1,0,0,0,0,0,0,NaN,19.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99211,2650,2019-08-01,1,NaN,NaN,NaN,0,0,0,0,0,1,0,1,0,0,0,0,0,0,NaN,21.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99212,2651,2019-08-01,1,NaN,2463.8,339.0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,NaN,14.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [980]:
# добавление фичей
cols_to_merge=['smena_1', 'smena_2', 'smena_3', 'sick_month',
       'work_time', 'sick_rate']
merged_data=merged_data.merge(train_full[cols_to_merge], how='left', left_on='hash_tab_num', right_on=train_full.index)

In [981]:
#merged_data.loc[merged_data.work_experience_company.isnull(), 'work_experience_company']=merged_data.work_experience_company.mean()
#merged_data.loc[merged_data.home_to_work_distance.isnull(), 'home_to_work_distance']=merged_data.home_to_work_distance.mean()
#merged_data.loc[merged_data.personel_num.isnull(), 'personel_num']=merged_data.personel_num.mean()

In [982]:
merged_data

,hash_tab_num,date,gender,work_experience_company,home_to_work_distance,personel_num,category_Рабочие,category_Руководители,category_Служащие,category_Специалисты,education_Высшее,education_Начальное_среднее,education_Среднее_профессинальное,razryad_fact_0,razryad_fact_1,razryad_fact_2,razryad_fact_3,razryad_fact_4,razryad_fact_5,razryad_fact_6,rel_is_male,work_shift_type_count,sick,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,y_10,y_11,y_12,smena_1,smena_2,smena_3,sick_month,work_time,sick_rate
0,0,2015-04-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,186.65,1505.5,106.7,15,53,0.283019
1,0,2015-05-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,186.65,1505.5,106.7,15,53,0.283019
2,0,2015-06-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,186.65,1505.5,106.7,15,53,0.283019
3,0,2015-07-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,186.65,1505.5,106.7,15,53,0.283019
4,0,2015-08-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,186.65,1505.5,106.7,15,53,0.283019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99209,2648,2019-08-01,1,NaN,4953.5,NaN,0,0,0,0,0,1,0,1,0,0,0,0,0,0,NaN,19.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.00,96.0,56.0,0,1,0.000000
99210,2649,2019-08-01,1,NaN,NaN,NaN,0,0,0,0,0,0,1,1,0,0,0,0,0,0,NaN,19.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.00,61.1,96.0,0,1,0.000000
99211,2650,2019-08-01,1,NaN,NaN,NaN,0,0,0,0,0,1,0,1,0,0,0,0,0,0,NaN,21.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,228.4,7.5,0,1,0.000000
99212,2651,2019-08-01,1,NaN,2463.8,339.0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,NaN,14.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.50,22.5,37.5,1,1,1.000000


In [983]:
# Деление на трейн и тест для последующего прогноза

train = merged_data[merged_data.date < pd.to_datetime('2018-08-01')]
submission_extra = merged_data[merged_data.date == pd.to_datetime('2018-08-01')]
submission_extra.index = range(len(submission_extra))

In [996]:
train

,hash_tab_num,date,gender,work_experience_company,home_to_work_distance,personel_num,category_Рабочие,category_Руководители,category_Служащие,category_Специалисты,education_Высшее,education_Начальное_среднее,education_Среднее_профессинальное,razryad_fact_0,razryad_fact_1,razryad_fact_2,razryad_fact_3,razryad_fact_4,razryad_fact_5,razryad_fact_6,rel_is_male,work_shift_type_count,sick,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,y_10,y_11,y_12,smena_1,smena_2,smena_3,sick_month,work_time,sick_rate
0,0,2015-04-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,186.65,1505.50,106.70,15,53,0.283019
1,0,2015-05-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,186.65,1505.50,106.70,15,53,0.283019
2,0,2015-06-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,186.65,1505.50,106.70,15,53,0.283019
3,0,2015-07-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,186.65,1505.50,106.70,15,53,0.283019
4,0,2015-08-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,186.65,1505.50,106.70,15,53,0.283019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98112,2445,2018-07-01,1,NaN,NaN,220.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,NaN,18.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,541.24,1017.80,502.90,0,3,0.000000
98115,2446,2018-07-01,1,NaN,NaN,241.0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,NaN,17.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,421.70,1251.87,369.88,0,3,0.000000
98118,2447,2018-07-01,1,NaN,NaN,319.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,NaN,10.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,670.25,753.74,406.50,0,3,0.000000
98121,2448,2018-07-01,1,NaN,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,NaN,22.0,0.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,517.52,1160.59,367.02,2,5,0.400000


In [1063]:
# Обучение модели 12 раз с осуществлением предсказания на 12 месяцев.

submission_extra_pred = pd.DataFrame()
for i in range(1,13):
    y_col_name = 'y_' + str(i) 
    X = train.dropna(subset=[y_col_name])\
    .drop(['y_1', 'y_2', 'y_3', 'y_4', 'y_5', 'y_6', 
        'y_7', 'y_8', 'y_9', 'y_10', 'y_11', 'y_12',
        'date', 'hash_tab_num'], axis = 1)
    
    X.fillna(0, inplace=True)
    
    y = train.dropna(subset=[y_col_name])[y_col_name]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, 
                                                        shuffle=True, stratify=y)

    X_submission_extra = submission_extra\
    .drop(['y_1', 'y_2', 'y_3', 'y_4', 'y_5', 'y_6', 
        'y_7', 'y_8', 'y_9', 'y_10', 'y_11', 'y_12',
        'date', 'hash_tab_num'], axis = 1)

    X_submission_extra.fillna(0, inplace=True)

    model = catb.CatBoostClassifier(iterations=350,silent=True, learning_rate=0.005)

    model.fit(X_train, y_train)

    
    p, r, thresholds = precision_recall_curve(y_test, model.predict_proba(X_test)[:,1])
    f1_scores = 2*r*p / (r+p)
    f1_scores = f1_scores[p > 0]
    th = thresholds[np.argmax(f1_scores)]
    report = classification_report(y_test, (model.predict_proba(X_test)[:,1] >= th).astype(int))
    print(report)
    submission_extra_pred[y_col_name] = (model.predict_proba(X_submission_extra)[:,1] >= th).astype(int)

submission_extra_pred['hash_tab_num'] = submission_extra['hash_tab_num']

              precision    recall  f1-score   support

         0.0       0.94      0.86      0.90     21587
         1.0       0.34      0.54      0.42      2790

    accuracy                           0.83     24377
   macro avg       0.64      0.70      0.66     24377
weighted avg       0.87      0.83      0.84     24377

              precision    recall  f1-score   support

         0.0       0.93      0.87      0.90     21442
         1.0       0.32      0.47      0.38      2798

    accuracy                           0.82     24240
   macro avg       0.62      0.67      0.64     24240
weighted avg       0.86      0.82      0.84     24240

              precision    recall  f1-score   support

         0.0       0.93      0.84      0.88     21302
         1.0       0.30      0.51      0.38      2812

    accuracy                           0.80     24114
   macro avg       0.61      0.68      0.63     24114
weighted avg       0.86      0.80      0.82     24114



<ipython-input-1063-fb9816cd6c9d>:31: RuntimeWarning: invalid value encountered in true_divide
  f1_scores = 2*r*p / (r+p)


              precision    recall  f1-score   support

         0.0       0.93      0.82      0.87     21215
         1.0       0.28      0.54      0.37      2810

    accuracy                           0.79     24025
   macro avg       0.61      0.68      0.62     24025
weighted avg       0.85      0.79      0.81     24025

              precision    recall  f1-score   support

         0.0       0.93      0.86      0.89     21044
         1.0       0.32      0.48      0.38      2804

    accuracy                           0.82     23848
   macro avg       0.62      0.67      0.64     23848
weighted avg       0.85      0.82      0.83     23848

              precision    recall  f1-score   support

         0.0       0.93      0.82      0.87     20868
         1.0       0.29      0.54      0.37      2791

    accuracy                           0.79     23659
   macro avg       0.61      0.68      0.62     23659
weighted avg       0.85      0.79      0.81     23659

              preci

<ipython-input-1063-fb9816cd6c9d>:31: RuntimeWarning: invalid value encountered in true_divide
  f1_scores = 2*r*p / (r+p)


              precision    recall  f1-score   support

         0.0       0.93      0.79      0.86     20037
         1.0       0.28      0.59      0.38      2709

    accuracy                           0.77     22746
   macro avg       0.61      0.69      0.62     22746
weighted avg       0.86      0.77      0.80     22746

              precision    recall  f1-score   support

         0.0       0.93      0.83      0.88     19870
         1.0       0.30      0.52      0.38      2701

    accuracy                           0.79     22571
   macro avg       0.61      0.68      0.63     22571
weighted avg       0.85      0.79      0.82     22571



In [1064]:
# Представление результата работы модели в плоский вид

one_line_sub = pd.DataFrame(columns = ['hash_tab_num','date','target'])
for i in range(1,13):
    temp_result = pd.DataFrame(columns = ['hash_tab_num','date','target'])
    temp_result['hash_tab_num'] = submission_extra_pred['hash_tab_num']
    temp_result['date'] = pd.to_datetime('2018-09-01') + pd.DateOffset(months=i-1)
    temp_result['target'] = submission_extra_pred['y_' + str(i)]
    one_line_sub = pd.concat([one_line_sub, temp_result], axis = 0)

In [1065]:
one_line_sub

,hash_tab_num,date,target
0,0,2018-09-01,1
1,1,2018-09-01,0
2,2,2018-09-01,0
3,3,2018-09-01,0
4,4,2018-09-01,0
...,...,...,...
2124,2462,2019-08-01,1
2125,2463,2019-08-01,0
2126,2464,2019-08-01,0
2127,2465,2019-08-01,0


In [1066]:
#получение y_val

In [1067]:
cols=['y_1','y_2','y_3','y_4','y_5','y_6','y_7','y_8', 'y_9','y_10', 'y_11','y_12', 'hash_tab_num']
y_val = merged_data[merged_data.date == pd.to_datetime('2018-09-01')][cols]


In [1068]:
y_val.fillna(0,inplace=True )

In [1069]:
# Представление результата работы модели в плоский вид

one_line_answer = pd.DataFrame(columns = ['hash_tab_num','date','target'])
for i in range(1,13):
    temp_result = pd.DataFrame(columns = ['hash_tab_num','date','target'])
    temp_result['hash_tab_num'] = y_val['hash_tab_num']
    temp_result['date'] = pd.to_datetime('2018-09-01') + pd.DateOffset(months=i-1)
    temp_result['target'] = y_val['y_' + str(i)]
    one_line_answer = pd.concat([one_line_answer, temp_result], axis = 0)

In [1070]:
one_line_sub=one_line_sub[one_line_sub.hash_tab_num.isin(one_line_answer.hash_tab_num.unique())]

In [1071]:
one_line_answer=one_line_answer[one_line_answer.hash_tab_num.isin(one_line_sub.hash_tab_num.unique())]

In [1072]:
one_line_sub['target']=one_line_sub.target.astype('float64')

<ipython-input-1072-0f5e64b30279>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_line_sub['target']=one_line_sub.target.astype('float64')


In [1073]:
report = classification_report(one_line_answer.target.values,one_line_sub.target.values)
print(report)

              precision    recall  f1-score   support

         0.0       0.93      0.83      0.88     22743
         1.0       0.25      0.48      0.33      2577

    accuracy                           0.80     25320
   macro avg       0.59      0.66      0.60     25320
weighted avg       0.86      0.80      0.82     25320



In [1074]:
f1_score(one_line_answer.target.values,one_line_sub.target.values)

0.32688002107204

In [1075]:
#0.32688002107204
#model.get_all_params()

{'nan_mode': 'Min',
 'eval_metric': 'Logloss',
 'iterations': 350,
 'sampling_frequency': 'PerTree',
 'leaf_estimation_method': 'Newton',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'bayesian_matrix_reg': 0.10000000149011612,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': False,
 'model_size_reg': 0.5,
 'subsample': 0.800000011920929,
 'use_best_model': False,
 'class_names': [0, 1],
 'random_seed': 0,
 'depth': 6,
 'posterior_sampling': False,
 'border_count': 254,
 'classes_count': 0,
 'auto_class_weights': 'None',
 'sparse_features_conflict_fraction': 0,
 'leaf_estimation_backtracking': 'AnyImprovement',
 'best_model_min_trees': 1,
 'model_shrink_rate': 0,
 'min_data_in_leaf': 1,
 'loss_function': 'Logloss',
 'learning_rate': 0.004999999888241291,
 'score_function': 'Cosine',
 'task_type': 'CPU',
 'leaf_estimation_iterations': 10,
 'boo

In [ ]:
#crop

In [862]:
# Подготовка файла submission
submission = pd.read_csv('data\submission_check.csv', sep = ';')
submission.date = pd.to_datetime(submission.date, format='%d.%m.%Y')
submission.drop('target', axis =1, inplace = True)
submission.head()

,hash_tab_num,date
0,0,2019-09-01
1,0,2019-10-01
2,0,2019-11-01
3,0,2019-12-01
4,0,2020-01-01


In [854]:
one_line_sub['hash_tab_num'] = pd.to_numeric(one_line_sub['hash_tab_num'])
submission_final = pd.merge(submission, one_line_sub, how = 'left', on = ['hash_tab_num','date'])


In [855]:
list_to_exclude

Int64Index([   2,   10,   13,   15,   24,   25,   29,   30,   32,   34,
            ...
            2628, 2634, 2635, 2636, 2637, 2638, 2647, 2649, 2650, 2652],
           dtype='int64', length=699)

In [856]:
submission_final.loc[submission_final.hash_tab_num.isin(list_to_exclude), 'target']=0

In [857]:
submission_final.target.value_counts()

0    16182
1     4839
Name: target, dtype: int64

In [858]:
submission_final.to_csv('3.csv', sep=';', index=False)